In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

RANDOM_STATE = 42

torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed_all(RANDOM_STATE)  # For multi-GPU.
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)


def _init_fn(worker_id):
    np.random.seed(int(RANDOM_STATE))

In [3]:
train = pd.read_csv('../data/train.csv', header=None, names=["text"])
test = pd.read_csv('../data/answers.csv', header=None, names=["text"])


def separate_xy(df, prefix="train"):
    df[prefix + "_X"] = df["text"].apply(lambda x: x[:-1])
    df[prefix + "_y"] = df["text"].apply(lambda x: x[-1])
    return df

train_df = separate_xy(train, 'train')
test_df = separate_xy(test, 'test')

In [4]:
train_df

,text,train_X,train_y
0,SR2SR2SGR,SR2SR2SG,R
1,SRGSRSRSR,SRGSRSRS,R
2,KFKFKFKFK,KFKFKFKF,K
3,X2LS2FRS2,X2LS2FRS,2
4,MMMMGGGGG,MMMMGGGG,G
...,...,...,...
2538,S2ZR22222,S2ZR2222,2
2539,62SNRM1MM,62SNRM1M,M
2540,ASSRSRSRS,ASSRSRSR,S
2541,2ZZT2Z2ZT,2ZZT2Z2Z,T


In [5]:
train_X, val_X, train_y, val_y = train_test_split(train_df["train_X"], train_df["train_y"], test_size=0.2, random_state=RANDOM_STATE)

In [6]:
vocab = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567'

id2char = {idx: char for idx, char in enumerate(vocab)}
char2id = {char: idx for idx, char in enumerate(vocab)}


# Define the size of the input (number of unique characters), hidden layer, and output (same as input)
input_size = len(vocab)
hidden_size = len(vocab) * 3
output_size = len(vocab)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NextCharCNN(nn.Module):
    def __init__(self, max_seq_length, vocab_size, num_output_classes, embedding_dim, num_conv_filters, conv_kernel_size):
        super(NextCharCNN, self).__init__()

        # Embedding parameters
        self.embedding_dim = embedding_dim  # Dimension of the embedding vector

        # Embedding layer to convert character indices into embeddings
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=self.embedding_dim)

        # Convolutional layer parameters
        self.num_conv_filters = num_conv_filters  # Number of filters in the convolutional layer
        self.conv_kernel_size = conv_kernel_size  # Size of the convolutional kernel

        # Calculate the size of the output from the convolutional layer
        # Assuming no padding ('valid' convolution)
        conv_output_length = max_seq_length - self.conv_kernel_size + 1

        # Convolutional layer to extract features from the embedded sequence
        self.conv_layer = nn.Conv1d(in_channels=self.embedding_dim, out_channels=self.num_conv_filters, kernel_size=self.conv_kernel_size)

        # Fully connected layer to map the convolution output to the number of output classes
        self.fc_layer = nn.Linear(in_features=self.num_conv_filters * conv_output_length, out_features=num_output_classes)

    def forward(self, x):
        # Embedding input (batch_size, max_seq_length) -> (batch_size, max_seq_length, embedding_dim)
        embedded_seq = self.embedding_layer(x)  

        # Permute to match Conv1d input format (batch_size, embedding_dim, max_seq_length)
        embedded_seq = embedded_seq.permute(0, 2, 1)

        # Apply convolution and ReLU activation (batch_size, num_conv_filters, conv_output_length)
        conv_output = F.relu(self.conv_layer(embedded_seq))

        # Flatten the convolution output to feed into the fully connected layer
        flattened_output = torch.flatten(conv_output, start_dim=1)

        # Fully connected layer output (batch_size, num_output_classes)
        fc_output = self.fc_layer(flattened_output)

        return fc_output

# Example of model initialization
max_seq_length = 8  # Maximum length of the input sequences
vocab_size = 33    # Size of the character set (vocabulary)
num_output_classes = 33  # Number of unique characters (same as vocab size for character prediction)
num_conv_filters = 16  # Number of filters in the convolutional layer
conv_kernel_size = 3  # Size of the convolutional kernel
embedding_dim = 100

model = NextCharCNN(max_seq_length, vocab_size, num_output_classes, embedding_dim, num_conv_filters, conv_kernel_size)
model

NextCharCNN(
  (embedding_layer): Embedding(33, 100)
  (conv_layer): Conv1d(100, 16, kernel_size=(3,), stride=(1,))
  (fc_layer): Linear(in_features=96, out_features=33, bias=True)
)

In [8]:
import torch

def test_character_predictor_cnn():
    # Model parameters
    max_seq_length = 8  # Maximum length of input sequences
    vocab_size = 33     # Size of the character set (vocabulary)
    num_output_classes = 33  # Number of unique characters (for character prediction)
    num_conv_filters = 16     # Number of filters in the convolutional layer
    conv_kernel_size = 3      # Size of the convolutional kernel
    batch_size = 10           # Number of sequences in a batch
    embedding_dim = 100

    # Create a test batch of dummy input sequences
    # Each element in a sequence is a random integer representing a character index
    dummy_input = torch.randint(low=0, high=vocab_size, size=(batch_size, max_seq_length))

    # Initialize the model
    model = NextCharCNN(max_seq_length, vocab_size, num_output_classes, embedding_dim, num_conv_filters, conv_kernel_size)

    # Perform a forward pass with the dummy input
    with torch.no_grad():
        dummy_output = model(dummy_input)

    # Check output dimensions
    assert dummy_output.shape == (batch_size, num_output_classes), f"Expected output shape to be {(batch_size, num_output_classes)}, but got {dummy_output.shape}"

    print("Test passed: Model output dimensions are correct.")

# Run the test
test_character_predictor_cnn()


Test passed: Model output dimensions are correct.


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

class CharSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        """
        Args:
            sequences (list of strings): List of character sequences.
            labels (list of strings): List of single characters (labels).
        """
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        # Convert the sequence and label to numeric format
        sequence = torch.tensor([char2id[char] for char in self.sequences[idx]], dtype=torch.long)
        label = torch.tensor(char2id[self.labels[idx]], dtype=torch.long)

        return sequence, label


In [10]:
train_X

350     22MSR22E
2330    SSSS1SSE
2218    RSMGGGMG
1655    222KK211
2453    S2GRS2GR
          ...   
1638    1111SRER
1095    111R11SR
1130    SSGSRSRS
1294    MZTKE2XZ
860     22MSR1SE
Name: train_X, Length: 2034, dtype: object

In [11]:
train_df["train_X"]

0       SR2SR2SG
1       SRGSRSRS
2       KFKFKFKF
3       X2LS2FRS
4       MMMMGGGG
          ...   
2538    S2ZR2222
2539    62SNRM1M
2540    ASSRSRSR
2541    2ZZT2Z2Z
2542    22MMMZTS
Name: train_X, Length: 2543, dtype: object

In [12]:
full_train_dataset = CharSequenceDataset(train_df["train_X"], train_df["train_y"])
train_dataset = CharSequenceDataset(list(train_X), list(train_y))
val_dataset = CharSequenceDataset(list(val_X), list(val_y))
test_dataset = CharSequenceDataset(test_df["test_X"], test_df["test_y"])

full_train_dataset, train_dataset, val_dataset, test_dataset

(<__main__.CharSequenceDataset at 0x7f39b4f37710>,
 <__main__.CharSequenceDataset at 0x7f39b4f37a10>)

In [13]:
# Define batch size
batch_size = 64

# Create the DataLoader for training data
full_train_loader = DataLoader(full_train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=_init_fn)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=_init_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=_init_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=_init_fn)

full_train_loader, train_loader, val_loader, test_loader

(<torch.utils.data.dataloader.DataLoader at 0x7f39b4f21150>,
 <torch.utils.data.dataloader.DataLoader at 0x7f39b4f44c90>)

In [14]:

def predict(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    actuals = []

    with torch.no_grad():
        for data, targets in data_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)

            predictions.extend(predicted.cpu().numpy())
            actuals.extend(targets.cpu().numpy())

    return predictions, actuals


In [15]:

def create_and_train_model(max_seq_length, vocab_size, num_output_classes, embedding_dim, num_conv_filters, conv_kernel_size, train_loader, val_loader, learning_rate, num_epochs, model_save_path):
    # Create the model instance
    model = NextCharCNN(max_seq_length, vocab_size, num_output_classes, embedding_dim, num_conv_filters, conv_kernel_size)
   
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Initialize the best validation loss to infinity at the start
    best_val_loss = float('inf')
    patience = 5  # Number of epochs to wait for improvement before stopping
    epochs_no_improve = 0  # Track the number of epochs with no improvement

    # Training function
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for batch_idx, (data, targets) in enumerate(train_loader):
            # Forward pass
            scores = model(data)
            loss = criterion(scores, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, targets in val_loader:
                scores = model(data)
                loss = criterion(scores, targets)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        # Print training and validation loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_save_path)
            print(f"Saved model with validation loss: {best_val_loss:.4f}")
            epochs_no_improve = 0  # Reset counter
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs.")
                break  # Early stopping

    # Return the trained model and its best validation loss
    return model, best_val_loss


In [16]:

def objective(trial):
    # Define the hyperparameter search space
    embedding_dim = trial.suggest_categorical('embedding_dim', [50, 100, 128])
    num_conv_filters = trial.suggest_categorical('num_conv_filters', [16, 32, 64])
    conv_kernel_size = trial.suggest_int('conv_kernel_size', 2, 5)
    lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    num_epochs = trial.suggest_int('num_epochs', 50, 100)

    # Train the model using the suggested hyperparameters
    model_save_path = f"../trained_models/cnn_model_trial_{trial.number}.pth"
    model, val_loss = create_and_train_model(max_seq_length, vocab_size, num_output_classes, embedding_dim, num_conv_filters, conv_kernel_size, train_loader, val_loader, lr, num_epochs, model_save_path)

    # Evaluate the model
    predictions, actuals = predict(model, val_loader)
    accuracy = accuracy_score(actuals, predictions)
    print(f"===== Accuracy {accuracy:.4f}.")

    return val_loss



In [17]:
n_trials=100

# Constants (could be parameterized as well)
max_seq_length = 8  # Assuming a fixed sequence length
vocab_size = 33     # Size of the character set (vocabulary)
num_output_classes = 33  # Number of unique characters


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)  # Set n_trials to the number of trials you want to run

best_trial = study.best_trial
print(f"Best trial: {best_trial.number}")
print(f"Best hyperparameters: {best_trial.params}")
print(f"Best validation loss: {best_trial.value}")

# Load the best model
best_model_path = f"../trained_models/cnn_model_trial_{best_trial.number}.pth"
{'embedding_dim': 128, 'num_conv_filters': 16, 'conv_kernel_size': 5, 'learning_rate': 0.00018717031657241407, 'num_epochs': 43}
best_model = NextCharCNN(
    max_seq_length, vocab_size, num_output_classes, best_trial.params['embedding_dim'], best_trial.params['num_conv_filters'], best_trial.params['conv_kernel_size']
)

best_model.load_state_dict(torch.load(best_model_path))


[I 2023-11-22 10:31:48,438] A new study created in memory with name: no-name-d1651271-1fb9-437d-ab96-b48fab64aedb
/tmp/ipykernel_85747/4040997793.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)


Epoch [1/90], Training Loss: 3.3135, Validation Loss: 3.0387
Saved model with validation loss: 3.0387
Epoch [2/90], Training Loss: 2.8221, Validation Loss: 2.6360
Saved model with validation loss: 2.6360
Epoch [3/90], Training Loss: 2.5633, Validation Loss: 2.4958
Saved model with validation loss: 2.4958
Epoch [4/90], Training Loss: 2.4511, Validation Loss: 2.4316
Saved model with validation loss: 2.4316
Epoch [5/90], Training Loss: 2.3768, Validation Loss: 2.3825
Saved model with validation loss: 2.3825
Epoch [6/90], Training Loss: 2.3165, Validation Loss: 2.3413
Saved model with validation loss: 2.3413
Epoch [7/90], Training Loss: 2.2553, Validation Loss: 2.3033
Saved model with validation loss: 2.3033
Epoch [8/90], Training Loss: 2.2029, Validation Loss: 2.2684
Saved model with validation loss: 2.2684
Epoch [9/90], Training Loss: 2.1542, Validation Loss: 2.2396
Saved model with validation loss: 2.2396
Epoch [10/90], Training Loss: 2.1077, Validation Loss: 2.2103
Saved model with val

[I 2023-11-22 10:31:53,247] Trial 0 finished with value: 1.996241107583046 and parameters: {'embedding_dim': 128, 'num_conv_filters': 32, 'conv_kernel_size': 5, 'learning_rate': 0.00017581921569191357, 'num_epochs': 90}. Best is trial 0 with value: 1.996241107583046.


Epoch [48/90], Training Loss: 1.4669, Validation Loss: 2.0014
Early stopping triggered after 48 epochs.
===== Accuracy 0.3929.
Epoch [1/82], Training Loss: 3.3917, Validation Loss: 3.2791
Saved model with validation loss: 3.2791


/tmp/ipykernel_85747/4040997793.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)


...

/tmp/ipykernel_85747/4040997793.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)


Epoch [4/79], Training Loss: 2.4354, Validation Loss: 2.3946
Saved model with validation loss: 2.3946
Epoch [5/79], Training Loss: 2.3473, Validation Loss: 2.3394
Saved model with validation loss: 2.3394
Epoch [6/79], Training Loss: 2.2768, Validation Loss: 2.3012
Saved model with validation loss: 2.3012
Epoch [7/79], Training Loss: 2.2179, Validation Loss: 2.2612
Saved model with validation loss: 2.2612
Epoch [8/79], Training Loss: 2.1604, Validation Loss: 2.2269
Saved model with validation loss: 2.2269
Epoch [9/79], Training Loss: 2.1090, Validation Loss: 2.1963
Saved model with validation loss: 2.1963
Epoch [10/79], Training Loss: 2.0679, Validation Loss: 2.1672
Saved model with validation loss: 2.1672
Epoch [11/79], Training Loss: 2.0214, Validation Loss: 2.1473
Saved model with validation loss: 2.1473
Epoch [12/79], Training Loss: 1.9812, Validation Loss: 2.1222
Saved model with validation loss: 2.1222
Epoch [13/79], Training Loss: 1.9462, Validation Loss: 2.1008
Saved model with 

[I 2023-11-22 10:37:38,320] Trial 99 finished with value: 1.997446447610855 and parameters: {'embedding_dim': 50, 'num_conv_filters': 32, 'conv_kernel_size': 5, 'learning_rate': 0.00035789677823217046, 'num_epochs': 79}. Best is trial 76 with value: 1.9660205692052841.


Epoch [40/79], Training Loss: 1.4345, Validation Loss: 2.0126
Early stopping triggered after 40 epochs.
===== Accuracy 0.4008.
Best trial: 76
Best hyperparameters: {'embedding_dim': 50, 'num_conv_filters': 32, 'conv_kernel_size': 5, 'learning_rate': 0.0002354780492317737, 'num_epochs': 77}
Best validation loss: 1.9660205692052841


<All keys matched successfully>

In [24]:
# Assuming 'model' is your trained SimpleRNN model
predictions, actuals = predict(best_model, val_loader)

# Print the classification report
print(classification_report(actuals, predictions, target_names=[id2char[s] for s in set(actuals + predictions)]))

# Calculate and print the accuracy
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy:.4f}")


              precision    recall  f1-score   support

           A       0.00      0.00      0.00         2
           C       0.25      0.12      0.17         8
           D       0.00      0.00      0.00         0
           E       0.21      0.19      0.20        32
           F       0.00      0.00      0.00         2
           G       0.45      0.40      0.42        52
           H       0.40      0.50      0.44         4
           I       0.33      0.22      0.27         9
           J       0.40      0.27      0.32        15
           K       0.46      0.52      0.49        23
           L       1.00      0.50      0.67         2
           M       0.58      0.67      0.62        27
           N       0.00      0.00      0.00         2
           P       0.47      0.47      0.47        49
           Q       1.00      1.00      1.00         1
           R       0.34      0.49      0.40        63
           S       0.33      0.43      0.38        86
           T       0.69    

/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

In [25]:
# Assuming 'model' is your trained SimpleRNN model
predictions, actuals = predict(best_model, test_loader)

# Print the classification report
print(classification_report(actuals, predictions, target_names=[id2char[s] for s in set(actuals + predictions)]))

# Calculate and print the accuracy
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy:.4f}")


              precision    recall  f1-score   support

           A       1.00      0.50      0.67         2
           C       0.00      0.00      0.00         0
           E       0.15      0.67      0.25         3
           F       0.00      0.00      0.00         0
           G       0.77      0.89      0.83        27
           H       0.00      0.00      0.00         0
           I       0.50      1.00      0.67         1
           J       0.86      0.67      0.75         9
           K       1.00      0.92      0.96        12
           M       0.88      0.82      0.85        28
           N       0.00      0.00      0.00         0
           P       0.98      0.75      0.85       111
           Q       0.00      0.00      0.00         0
           R       0.85      0.80      0.83        92
           S       0.55      0.75      0.63        24
           T       1.00      1.00      1.00        14
           W       0.00      0.00      0.00         0
           Z       0.00    

/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samp

In [26]:

embedding_dim = 128
lr = 0.0003
num_conv_filters = 64
num_channels = 16
kernel_size = 4
num_epochs = 55

embedding_dim = 50
num_conv_filters = 32
conv_kernel_size = 5
learning_rate =0.0002354780492317737
num_epochs = 77

model_save_path = f"../trained_models/model_trial_full_run_next_char_loss.pth"

model, val_loss = create_and_train_model(
    max_seq_length, vocab_size, num_output_classes, embedding_dim, num_conv_filters, conv_kernel_size, full_train_loader, test_loader, lr, num_epochs, model_save_path
)




Epoch [1/77], Training Loss: 3.2781, Validation Loss: 2.7626
Saved model with validation loss: 2.7626
Epoch [2/77], Training Loss: 2.7446, Validation Loss: 2.1563
Saved model with validation loss: 2.1563
Epoch [3/77], Training Loss: 2.5141, Validation Loss: 1.9352
Saved model with validation loss: 1.9352
Epoch [4/77], Training Loss: 2.4163, Validation Loss: 1.8018
Saved model with validation loss: 1.8018
Epoch [5/77], Training Loss: 2.3422, Validation Loss: 1.6830
Saved model with validation loss: 1.6830
Epoch [6/77], Training Loss: 2.2810, Validation Loss: 1.5828
Saved model with validation loss: 1.5828
Epoch [7/77], Training Loss: 2.2308, Validation Loss: 1.5447
Saved model with validation loss: 1.5447
Epoch [8/77], Training Loss: 2.1768, Validation Loss: 1.4609
Saved model with validation loss: 1.4609
Epoch [9/77], Training Loss: 2.1341, Validation Loss: 1.4023
Saved model with validation loss: 1.4023
Epoch [10/77], Training Loss: 2.0950, Validation Loss: 1.3092
Saved model with val

In [28]:
# Assuming 'model' is your trained SimpleRNN model
predictions, actuals = predict(best_model, test_loader)

# Print the classification report
print(classification_report(actuals, predictions, target_names=[id2char[s] for s in set(actuals + predictions)]))

# Calculate and print the accuracy
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy:.4f}")


              precision    recall  f1-score   support

           A       1.00      0.50      0.67         2
           C       0.00      0.00      0.00         0
           E       0.15      0.67      0.25         3
           F       0.00      0.00      0.00         0
           G       0.77      0.89      0.83        27
           H       0.00      0.00      0.00         0
           I       0.50      1.00      0.67         1
           J       0.86      0.67      0.75         9
           K       1.00      0.92      0.96        12
           M       0.88      0.82      0.85        28
           N       0.00      0.00      0.00         0
           P       0.98      0.75      0.85       111
           Q       0.00      0.00      0.00         0
           R       0.85      0.80      0.83        92
           S       0.55      0.75      0.63        24
           T       1.00      1.00      1.00        14
           W       0.00      0.00      0.00         0
           Z       0.00    

/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/giyaseddin/miniconda3/envs/chatbot-special-language-modelling-assignment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samp